# GNN Modeling

## Data and Set up

In [1]:
import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl

from torch_geometric.data import Data

from tqdm import tqdm

In [2]:
# load edge list
edge_list_path = 'data/edge_list.npy'
edge_list = torch.Tensor(np.load(edge_list_path).T).type(torch.int64) # read in format expected by pytorch geometric [2, n_edges]

# load protein-ID dictionary (need new ID system starting at index 0 for pytorch geometric)
protein_id_dict = np.load('data/protein_ids_dict.npy', allow_pickle=True).item() # maps my custom ID system to Ensembl IDs
protein_id_dict_inv = {Ensembl: id_ for id_, Ensembl in protein_id_dict.items()} # maps Ensembl IDs to my custom ID system

In [3]:
data_path = 'data/HPAnode_PPInetwork_labels_v2.csv' #NOTE: labels need to be generated from infomation in this dataset (FIXME)
node_dataset = pd.read_csv(data_path, index_col=0)

# map dataset
myID = node_dataset.index.map(protein_id_dict_inv).rename('myID')
node_dataset.insert(loc=0, column='myID', value=myID)
node_dataset = node_dataset.reset_index().set_index('myID')

In [4]:
# make sure dataset with myID is of correct order and format
node_dataset.sort_index(inplace=True) # should already be sorted, but just in case
assert((node_dataset.index.to_numpy() == np.arange(len(node_dataset))).all())

In [5]:
# create positives
label_name = 'my_label'

# find positives
pos_label_col = 'Any_pos' #FIXME: figure out meaning of columns and determing appropriate choice of positive labels
pos_labels = pd.array([1 if row[pos_label_col] else None for id_, row in node_dataset.iterrows()], dtype='Int32')
node_dataset[label_name] = pos_labels

# create negatives
def sample_negatives(PU_labels):
    '''randomly samples from the unlabeled samples'''

    # sample same # as positives
    num_pos = (PU_labels==1).sum()
    neg_inds = PU_labels[PU_labels.isna()].sample(num_pos).index

    # TODO: more sophisticated methods for sampling methods. (e.g.: use mutation rate, unsupervised learning, etc.)

    return neg_inds # returns ID's of negative samples

neg_label_inds = sample_negatives(node_dataset[label_name])
node_dataset[label_name].loc[neg_label_inds] = 0

# TODO: save this data for reproducibility (not now, but once this is finalized and fixed)

node_dataset[label_name].value_counts()

0    1268
1    1268
Name: my_label, dtype: Int64

In [6]:
label_col = label_name
node_dataset[label_col] = node_dataset[label_col].astype('Int32')

# TODO: decide whether or not to include network embedding features...
num_node_feats = 100
node_feat_cols = ['Tissue RNA - lung [NX]', 'Single Cell Type RNA - Mucus-secreting cells [NX]'] + [f'node_{i}' for i in range(num_node_feats)]

# get subset of node features features + labels
node_data = node_dataset[node_feat_cols + [label_col]]

X = torch.Tensor(node_data[node_feat_cols].to_numpy())#.type(torch.float64)

y = node_data[label_col].fillna(-1).astype('int') # fill NaN with -1 so that it can be converted to pytorch tensor
y = torch.Tensor(y).type(torch.int64)

# restrict to data with labels
node_data_labeled = node_data[node_data[label_col].notna()]
node_data_labeled

,Tissue RNA - lung [NX],Single Cell Type RNA - Mucus-secreting cells [NX],node_0,node_1,node_2,node_3,node_4,node_5,node_6,node_7,...,node_91,node_92,node_93,node_94,node_95,node_96,node_97,node_98,node_99,my_label
myID,,,,,,,,,,,,,,,,,,,,,
0,-0.293745,-0.037880,1.127839,0.280114,-0.562910,0.680988,-0.107089,0.072526,0.313232,0.584403,...,-0.024280,-0.001387,0.020212,0.003985,0.038392,-0.000400,0.005882,0.026843,-0.021281,0
2,-0.113110,-0.085092,0.917932,0.107147,-0.434965,-0.383316,0.318524,-0.244370,-0.418846,0.215204,...,-0.064491,-0.112350,0.170609,-0.053770,0.045259,0.019909,-0.247172,-0.091207,0.132493,0
5,-0.154398,-0.105079,0.988793,0.956698,1.227737,-0.472868,-0.674905,1.143741,-0.281922,-0.545074,...,0.002763,0.020379,-0.005631,-0.036373,0.017688,0.059181,0.044308,-0.040594,-0.096454,0
13,-0.427931,-0.086981,2.043737,1.748582,1.079505,0.433287,-0.009460,0.671909,-0.519080,-0.146116,...,0.124833,0.008721,0.032194,-0.150306,0.071645,-0.257054,0.229697,0.140799,-0.118054,0
14,-0.175042,-0.068096,1.354122,0.340012,-0.965181,-0.526789,0.439390,0.140608,0.423648,0.819132,...,-0.010304,0.042092,0.023966,0.015752,-0.029377,-0.084104,0.018157,-0.005635,0.040090,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14531,-0.567278,-0.102876,1.015411,-0.072371,0.365560,-0.138961,-0.347684,-0.435399,-0.253599,0.219127,...,-0.032206,0.024851,-0.013315,-0.006128,-0.015564,0.020981,-0.027659,-0.013440,-0.017793,0
14532,-0.495024,-0.100515,1.191883,0.193004,0.741723,-0.290044,-0.536447,-0.683263,-0.090337,0.166989,...,-0.033623,0.028087,-0.021190,0.000559,-0.017235,0.022398,-0.042681,-0.006727,-0.020374,0
14541,-0.546634,-0.105079,1.425105,-0.314999,-0.784846,0.288170,-0.228677,-0.068500,0.441017,-0.260978,...,0.004942,-0.000126,0.029034,-0.011315,0.000936,-0.009829,-0.005649,0.001292,-0.005044,0


In [7]:
from sklearn.model_selection import train_test_split

X_myIDs = node_data_labeled.index.to_numpy() # myIDs for nodes with labels for training/testing
labels = node_data_labeled[label_col].to_numpy() # for stratification

test_size = 0.2
val_size = 0.1 * (1/(1-test_size))

myIDs_train_val, myIDs_test = train_test_split(X_myIDs, test_size=test_size, shuffle=True, stratify=labels)

labels_train_val = node_data_labeled.loc[myIDs_train_val][label_col].to_numpy()
myIDs_train, myIDs_val = train_test_split(myIDs_train_val, test_size=val_size, shuffle=True, stratify=labels_train_val)

# NOTE: train-val-test split is shuffled and stratified
# TODO: look into any special consideration necessary for train-test splits on graph-based models

# create masks
n_nodes = len(node_data)
train_mask = np.zeros(n_nodes, dtype=bool)
train_mask[myIDs_train] = True
train_mask = torch.Tensor(train_mask).type(torch.bool)

val_mask = np.zeros(n_nodes, dtype=bool)
val_mask[myIDs_val] = True
val_mask = torch.Tensor(val_mask).type(torch.bool)

test_mask = np.zeros(n_nodes, dtype=bool)
test_mask[myIDs_test] = True
test_mask = torch.Tensor(test_mask).type(torch.bool)

In [8]:
data = Data(x=X, y=y, edge_index=edge_list)
num_classes = 2
num_features = X.shape[1]

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

## Graph Convolutional Neural Network

In [9]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# define GCN architecture
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers, dropout_rate=0):
        super(GCN, self).__init__()
        self.convs = []
        self.convs.append(GCNConv(num_features, hidden_channels)) # first GCNConv layer

        for _ in range(num_layers - 1): # middle layers
            self.convs.append(GCNConv(hidden_channels, hidden_channels))

        self.convs = torch.nn.ModuleList(self.convs)

        self.dense1 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.dense_out = torch.nn.Linear(hidden_channels, num_classes)

        self.dropout_rate = dropout_rate

    def forward(self, x, edge_index):

        for conv in self.convs:
            x = conv(x, edge_index)
            x = x.relu()
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

        x = self.dense1(x)
        x = x.relu()
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = self.dense_out(x)

        return x

model = GCN(hidden_channels=16, num_layers=10, dropout_rate=0)
print(model)

GCN(
  (convs): ModuleList(
    (0): GCNConv(102, 16)
    (1): GCNConv(16, 16)
    (2): GCNConv(16, 16)
    (3): GCNConv(16, 16)
    (4): GCNConv(16, 16)
    (5): GCNConv(16, 16)
    (6): GCNConv(16, 16)
    (7): GCNConv(16, 16)
    (8): GCNConv(16, 16)
    (9): GCNConv(16, 16)
  )
  (dense1): Linear(in_features=16, out_features=16, bias=True)
  (dense_out): Linear(in_features=16, out_features=2, bias=True)
)


In [10]:
import pytorch_lightning as pl

# define Pytorch Lightning model
class LitGCN(pl.LightningModule):
    def __init__(self, model_name, **model_kwargs):
        super().__init__()
        # Saving hyperparameters
        self.save_hyperparameters()

        self.model_name = model_name
        self.model = GCN(**model_kwargs)
        self.loss_module = torch.nn.CrossEntropyLoss()

        self.example_input_array = data

    def forward(self, data, mode="train"):
        x, edge_index = data.x, data.edge_index
        x = self.model(x, edge_index)

        # Only calculate the loss and acc on the nodes corresponding to the mask
        if mode == "train":
            mask = data.train_mask
        elif mode == "val":
            mask = data.val_mask
        elif mode == "test":
            mask = data.test_mask
        else:
            assert False, "Unknown forward mode: %s" % mode

        #TODO: add other metrics like recall, precision, f1, etc...
        loss = self.loss_module(x[mask], data.y[mask])
        acc = (x[mask].argmax(dim=-1) == data.y[mask]).sum().float() / mask.sum()
        return x, loss, acc

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())#SGD(self.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, loss, acc = self.forward(batch, mode="train")
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_acc", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        logits, _, acc = self.forward(batch, mode="val")
        self.log("val_acc", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return logits

    def validation_epoch_end(self, validation_step_outputs):
        # NOTE: can't save non-standard GNN model like this
        # TODO: look into how to save torch geometric models
        # dummy_input = data
        # model_filename = f'{self.model_name}_{str(self.global_step).zfill(5)}.onnx'
        # torch.onnx.export(self, dummy_input, model_filename)
        # wandb.save(model_filename)

        flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
        self.logger.experiment.log({'val_logits': wandb.Histogram(flattened_logits.to('cpu')), 
                                    'global_step': self.global_step})

    def test_step(self, batch, batch_idx):
        x, _, acc = self.forward(batch, mode="test")
        self.log("test_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)

    # def test_epoch_end(self, test_step_outputs):
    #     # save model as onnx format
    #     pass

In [11]:
import os
notebook_name = 'modeling_gnn.ipynb'
os.environ['WANDB_NOTEBOOK_NAME'] = notebook_name

In [12]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
import wandb
import torch_geometric.loader

model_name = 'my_gcn_test3'

# logger = TensorBoardLogger("tb_logs", name=model_name)#, log_graph=True)
logger = WandbLogger(name=model_name, project="Project X", log_model="all")#, version=...)


AVAIL_GPUS = min(1, torch.cuda.device_count())
# AVAIL_GPUS = 0 # use when running out VRAM

model = LitGCN(model_name, hidden_channels=16, num_layers=2)#hidden_channels=64, num_layers=10, dropout_rate=0)

data_loader = torch_geometric.loader.DataLoader([data])#, batch_size=1, num_workers=2)


trainer = pl.Trainer(
        callbacks=[ModelCheckpoint(save_weights_only=False, mode="max", monitor="val_acc")],
        gpus=AVAIL_GPUS,
        max_epochs=50,
        logger=logger,
        # progress_bar_refresh_rate=0,
    )  # 0 because epoch size is 1

trainer.fit(model, data_loader, data_loader)
model = LitGCN.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: Currently logged in as: awni00 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name        | Type             | Params | In sizes                     | Out sizes 
---------------------------------------------------------------------------------------------
0 | model       | GCN              | 2.2 K  | [[14552, 102], [2, 4214097]] | [14552, 2]
1 | loss_module | CrossEntropyLoss | 0      | [[1774, 2], [1774]]          | ?         
---------------------------------------------------------------------------------------------
2.2 K     Trainable params
0         Non-trainable params
2.2 K     Total params
0.009     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\awnya\miniconda3\envs\projectx\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


C:\Users\awnya\miniconda3\envs\projectx\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\awnya\miniconda3\envs\projectx\lib\site-packages\pytorch_lightning\trainer\data_loading.py:326: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 49: 100%|██████████| 2/2 [00:00<00:00,  6.29it/s, loss=0.672, v_num=s8k6, train_loss_step=0.663, train_acc_step=0.639, val_acc_step=0.638, val_acc_epoch=0.638, train_loss_epoch=0.664, train_acc_epoch=0.639]


In [13]:
# evaluate

from sklearn.metrics import classification_report
logits, _, _ = model.forward(data.to(device='cpu'))

preds_train = logits[data.train_mask].argmax(dim=-1)
preds_test = logits[data.test_mask].argmax(dim=-1)

y_train = data.y[data.train_mask]
y_test = data.y[data.test_mask]

train_report = classification_report(y_train, preds_train, labels=[0,1], target_names=['negative', 'positive'])
test_report = classification_report(y_test, preds_test, labels=[0,1], target_names=['negative', 'positive'])

print('training metrics')
print(train_report)
print()
print('testing metrics')
print(test_report)

training metrics
              precision    recall  f1-score   support

    negative       0.64      0.63      0.64       887
    positive       0.64      0.65      0.64       887

    accuracy                           0.64      1774
   macro avg       0.64      0.64      0.64      1774
weighted avg       0.64      0.64      0.64      1774


testing metrics
              precision    recall  f1-score   support

    negative       0.63      0.65      0.64       254
    positive       0.64      0.62      0.63       254

    accuracy                           0.64       508
   macro avg       0.64      0.64      0.64       508
weighted avg       0.64      0.64      0.64       508

